In [12]:
# import libraries
import pandas as pd 
import numpy as np 
import seaborn as sns 
import matplotlib.pyplot as plt 
import calendar 
import datetime as dt  
%matplotlib inline 

In [13]:
# read in customer_demographic sheet from file
customer_demographic = pd.read_excel('D:\OneDrive\Máy tính\KPMG_VI_New_raw_data_update_final.xlsx\KPMG_VI_New_raw_data_update_final.xlsx', sheet_name='CustomerDemographic', header=1)

In [14]:
customer_demographic

,customer_id,first_name,last_name,gender,past_3_years_bike_related_purchases,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,default,owns_car,tenure
0,1,Laraine,Medendorp,F,93,1953-10-12 00:00:00,Executive Secretary,Health,Mass Customer,N,"""'",Yes,11.0
1,2,Eli,Bockman,Male,81,1980-12-16 00:00:00,Administrative Officer,Financial Services,Mass Customer,N,<script>alert('hi')</script>,Yes,16.0
2,3,Arlin,Dearle,Male,61,1954-01-20 00:00:00,Recruiting Manager,Property,Mass Customer,N,2018-02-01 00:00:00,Yes,15.0
3,4,Talbot,NaN,Male,33,1961-10-03 00:00:00,NaN,IT,Mass Customer,N,() { _; } >_[$($())] { touch /tmp/blns.shellsh...,No,7.0
4,5,Sheila-kathryn,Calton,Female,56,1977-05-13 00:00:00,Senior Editor,NaN,Affluent Customer,N,NIL,Yes,8.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3995,3996,Rosalia,Halgarth,Female,8,1975-08-09 00:00:00,VP Product Management,Health,Mass Customer,N,-100,No,19.0
3996,3997,Blanch,Nisuis,Female,87,2001-07-13 00:00:00,Statistician II,Manufacturing,High Net Worth,N,â¦testâ§,Yes,1.0
3997,3998,Sarene,Woolley,U,60,NaN,Assistant Manager,IT,High Net Worth,N,NaN,No,NaN
3998,3999,Patrizius,NaN,Male,11,1973-10-24 00:00:00,NaN,Manufacturing,Affluent Customer,N,Â¡â¢Â£Â¢âÂ§Â¶â¢ÂªÂºââ,Yes,10.0


In [15]:
# check summary statistics
customer_demographic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4000 entries, 0 to 3999
Data columns (total 13 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   customer_id                          4000 non-null   int64  
 1   first_name                           4000 non-null   object 
 2   last_name                            3875 non-null   object 
 3   gender                               4000 non-null   object 
 4   past_3_years_bike_related_purchases  4000 non-null   int64  
 5   DOB                                  3913 non-null   object 
 6   job_title                            3494 non-null   object 
 7   job_industry_category                3344 non-null   object 
 8   wealth_segment                       4000 non-null   object 
 9   deceased_indicator                   4000 non-null   object 
 10  default                              3698 non-null   object 
 11  owns_car                      

Intital Observations:

After examining the dataframe, the identified issues are:

past_3_years_bike_related_purchases,tenure is stored as object, instaed of numerical format, potentially affecting data analysis.

DOB is stored as object where it should be converted to datetime.

Multiple columns have missing values that need addressing before analysis: last_name,DOB,job_title,job_industry_category,default,tenure.

Oldest customer born in 1843, identified as 179 years old, showing a lack of consistency.

Only 2 out of 4000 customers identified as deceased.

Gender column values: Male(1872), M(1), Female(2037), F/Femal(2), Unisexual(U)(88) lacks consistency.

Phik correlation shows no correlation between most variables, except for Gender and Job Industry.

Features Engineering:

The following strategies can be used to handle missing data:

Fill missing values with the mean for continuous variables without outliers.

Fill missing values with the median for continuous variables with outliers.

Fill missing values with the mode for categorical variables.

Use forward-fill (ffill) or backward-fill (bfill) methods to fill missing data.

Drop the entire variable if there are a large number of missing data points.

Drop rows containing null values if they won't significantly impact the analysis.

In [16]:
# count missing values in each variable
customer_demographic.isnull().sum()

customer_id                              0
first_name                               0
last_name                              125
gender                                   0
past_3_years_bike_related_purchases      0
DOB                                     87
job_title                              506
job_industry_category                  656
wealth_segment                           0
deceased_indicator                       0
default                                302
owns_car                                 0
tenure                                  87
dtype: int64

In [17]:
# calculate missing value percentage for each column
customer_demographic.isnull().mean() * 100

customer_id                             0.000
first_name                              0.000
last_name                               3.125
gender                                  0.000
past_3_years_bike_related_purchases     0.000
DOB                                     2.175
job_title                              12.650
job_industry_category                  16.400
wealth_segment                          0.000
deceased_indicator                      0.000
default                                 7.550
owns_car                                0.000
tenure                                  2.175
dtype: float64

Missing Data:

Steps to address missing data:

last_name: has just 3.1% missing values, so we fill nulls with mode.

DOB: has 2.2% so we fill nulls with mode.

job_title: has 12.6% we will fill these values with mode.

job_industry_category: has 16.4% so we fill nulls with mode.

default: has 7.6% so we fill nulls with mode.

tenure: has 2.2% missing from the column so we will fill it with mean.

In [18]:
customer_demographic = customer_demographic.drop(columns=['default'])

In [19]:
# fill missing values in the 'follwoing' columns with the mode value
customer_demographic['last_name'] = customer_demographic['last_name'].fillna(customer_demographic['last_name'].mode()[0])
customer_demographic['DOB'] = customer_demographic['DOB'].fillna(customer_demographic['DOB'].mode()[0])
customer_demographic['job_title'] = customer_demographic['job_title'].fillna(customer_demographic['job_title'].mode()[0])
customer_demographic['job_industry_category'] = customer_demographic['job_industry_category'].fillna(customer_demographic['job_industry_category'].mode()[0])
# fill missing values in the 'tenure' columns with the mean value
customer_demographic['tenure'] = customer_demographic['tenure'].fillna(customer_demographic['tenure'].mean())

In [38]:
# check results of updated dataframe
customer_demographic.isnull().sum()

customer_id                            0
first_name                             0
last_name                              0
gender                                 0
past_3_years_bike_related_purchases    0
DOB                                    0
job_title                              0
job_industry_category                  0
wealth_segment                         0
deceased_indicator                     0
owns_car                               0
tenure                                 0
age                                    0
age_group                              0
dtype: int64

In [21]:
# amend column data types
customer_demographic['past_3_years_bike_related_purchases']=customer_demographic['past_3_years_bike_related_purchases'].astype('int')
customer_demographic['tenure']=customer_demographic['tenure'].astype('float')
customer_demographic['DOB']=customer_demographic['DOB'].astype('datetime64[ns]')

In [22]:
# check for duplicated rows
customer_demographic[customer_demographic.duplicated()]

,customer_id,first_name,last_name,gender,past_3_years_bike_related_purchases,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,owns_car,tenure


In [23]:
# check unique values of all columns
column_names = customer_demographic.columns
for i in column_names:
  print((i, customer_demographic[i].is_unique))

('customer_id', True)
('first_name', False)
('last_name', False)
('gender', False)
('past_3_years_bike_related_purchases', False)
('DOB', False)
('job_title', False)
('job_industry_category', False)
('wealth_segment', False)
('deceased_indicator', False)
('owns_car', False)
('tenure', False)


In [24]:
# count unique values of 'gender' column
customer_demographic['gender'].value_counts()

gender
Female    2037
Male      1872
U           88
F            1
Femal        1
M            1
Name: count, dtype: int64

In [39]:
# count unique values of 'gender' column
customer_demographic['age_group'].value_counts()

age_group
31-50    2002
51-70    1312
20-30     591
71-92       5
Name: count, dtype: int64

In [25]:
# replace inconsistent values with appropriate values
customer_demographic['gender'] = customer_demographic['gender'].replace('F','Female').replace('M','Male').replace('Femal','Female').replace('U','Unspecified')
customer_demographic['gender'].value_counts()

gender
Female         2039
Male           1873
Unspecified      88
Name: count, dtype: int64

In [26]:
# drop unspecified values of gender column 
customer_demographic.drop(customer_demographic[customer_demographic['gender'] == 'Unspecified'].index, inplace=True)
customer_demographic

,customer_id,first_name,last_name,gender,past_3_years_bike_related_purchases,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,owns_car,tenure
0,1,Laraine,Medendorp,Female,93,1953-10-12,Executive Secretary,Health,Mass Customer,N,Yes,11.0
1,2,Eli,Bockman,Male,81,1980-12-16,Administrative Officer,Financial Services,Mass Customer,N,Yes,16.0
2,3,Arlin,Dearle,Male,61,1954-01-20,Recruiting Manager,Property,Mass Customer,N,Yes,15.0
3,4,Talbot,Pristnor,Male,33,1961-10-03,Business Systems Development Analyst,IT,Mass Customer,N,No,7.0
4,5,Sheila-kathryn,Calton,Female,56,1977-05-13,Senior Editor,Manufacturing,Affluent Customer,N,Yes,8.0
...,...,...,...,...,...,...,...,...,...,...,...,...
3994,3995,Rusty,Iapico,Male,93,1975-12-12,Staff Scientist,Manufacturing,Mass Customer,N,Yes,14.0
3995,3996,Rosalia,Halgarth,Female,8,1975-08-09,VP Product Management,Health,Mass Customer,N,No,19.0
3996,3997,Blanch,Nisuis,Female,87,2001-07-13,Statistician II,Manufacturing,High Net Worth,N,Yes,1.0
3998,3999,Patrizius,Pristnor,Male,11,1973-10-24,Business Systems Development Analyst,Manufacturing,Affluent Customer,N,Yes,10.0


In [27]:
# check unique values of 'DOB' column
customer_demographic['DOB'].describe()

count                             3912
mean     1977-07-25 11:13:15.092024544
min                1931-10-23 00:00:00
25%                1968-01-25 00:00:00
50%                1977-07-25 00:00:00
75%                1987-02-28 06:00:00
max                2002-03-11 00:00:00
Name: DOB, dtype: object

In [28]:
# check unique values of 'deceased_indicator' column
customer_demographic['deceased_indicator'].unique()

array(['N', 'Y'], dtype=object)

In [29]:
# count unique values of 'deceased_indicator' column
customer_demographic['deceased_indicator'].value_counts()

deceased_indicator
N    3910
Y       2
Name: count, dtype: int64

In [30]:
# drop all rows where 'deceased_indicator' is equal to 'Y',
customer_demographic.drop(customer_demographic[customer_demographic['deceased_indicator'] == 'Y'].index, inplace=True)

In [31]:
# count unique values of 'deceased_indicator' column
customer_demographic['deceased_indicator'].value_counts()

deceased_indicator
N    3910
Name: count, dtype: int64

In [32]:
# function converts given date to age
def from_dob_to_age(born):
    today = dt.date.today()
    return today.year - born.year - ((today.month, today.day) < (born.month, born.day))

In [33]:
# apply function on the DOB column
customer_demographic['age']=customer_demographic['DOB'].apply(lambda x: from_dob_to_age(x))

In [36]:
# check unique values of 'DOB' column
customer_demographic['age'].describe()

count    3910.000000
mean       45.954987
std        12.617076
min        21.000000
25%        36.000000
50%        46.000000
75%        55.000000
max        92.000000
Name: age, dtype: float64

In [34]:
# divide age group
Age_group = pd.cut(customer_demographic.age, bins = [20,30,50,70,92], labels= ['20-30','31-50','51-70','71-92'])
customer_demographic.insert(13, 'age_group', Age_group)

In [35]:
customer_demographic.to_csv(r"d:\OneDrive\Máy tính\Segmentation\customer_demographic.csv", index=False)